In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt



In [6]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from typing import List
from typing import Tuple

ImportError: DLL load failed while importing _fblas: %1 is not a valid Win32 application.

In [8]:
df = pd.read_csv('Credit_Card.csv')
label_df = pd.read_csv('Credit_card_label.csv')
df.head()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2


In [7]:
df.columns


Index(['Ind_ID', 'GENDER', 'Car_Owner', 'Propert_Owner', 'CHILDREN',
       'Annual_income', 'Type_Income', 'EDUCATION', 'Marital_status',
       'Housing_type', 'Birthday_count', 'Employed_days', 'Mobile_phone',
       'Work_Phone', 'Phone', 'EMAIL_ID', 'Type_Occupation', 'Family_Members'],
      dtype='object')

# Variables to use:
Propert_owner,
Annual_income,
EDUCATION,
Housing_type,
Marital_status,
Children

# 1. (5 pts) Clean your dataset to turn categorical values into numerical ones.

In [12]:
Income = df.Annual_income.to_numpy()
Children = df.CHILDREN.to_numpy()

In [11]:
def to_pyint(list):
    dummylist = []
    newlist = []
    for item in list:
        try: dummylist.append(int(item.item()))
        except: pass
    listmean = mt.floor((np.mean(dummylist)).item())
    for item in list:
        try: newlist.append(int(item.item()))
        except: newlist.append(listmean)
    return newlist
def to_pyfloat(list):
    dummylist = []
    newlist = []
    for item in list:
        try: dummylist.append(float(item.item()))
        except: pass
    listmean = (np.mean(dummylist)).item()
    for item in list:
        try: newlist.append(float(item.item()))
        except: newlist.append(listmean)
    return newlist
def encoded(Dataf , list):
    dfnew = pd.get_dummies(Dataf[list])
    return df

In [13]:
sorted_df = pd.DataFrame({"Id" : label_df["Ind_ID"],
                       "Property": df["Propert_Owner"],
                       "Income": to_pyint(Income),
                       "Education": df["EDUCATION"],
                       "Housing": df["Housing_type"],
                       "Married": df["Marital_status"],
                       "Children": to_pyint(Children)})



Decided not to use one-hot encoding because I want to assign the semi-ordinal values my own order

In [52]:
sorted_df["Property"] = sorted_df["Property"].replace({"Y" : 1,
                                                      "N": 0}).values
sorted_df["Education"] = sorted_df["Education"].replace({"Lower secondary" : 0,
                                                         'Secondary / secondary special': 1,
                                                        'Incomplete higher' : 2,
                                                         'Higher education': 3,
                                                          'Academic degree' : 4 }).values
sorted_df["Married"] = sorted_df["Married"].replace({'Single / not married' : 0,
                                                         'Separated': 1,
                                                        'Civil marriage' : 2,
                                                         'Married': 3,
                                                          'Widow' : 4 }).values
sorted_df["Housing"] = sorted_df["Housing"].replace({'With parents': 0,
                                                         'Municipal apartment': 1,
                                                        'Rented apartment' : 2,
                                                         'Co-op apartment': 3,
                                                          'Office apartment' : 4,
                                                           'House / apartment': 5 }).values


(3 pts) Perform univariate linear regression on the dataset. Select your variable to predict. How well did this model perform? Is this a good approach for this dataset? Why or why not? 

In [ ]:
def credit_approve(sorted_df, 
                    xcolumn, 
                    ycolumn):

    X = sorted_df[xcolumn]
    y = sorted_df[ycolumn]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    y_pred = lin_reg.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mse, r2

In [ ]:
xcolumn = 'Income'
ycolumn = 'ID'

mse, r2 = credit_approve(sorted_df, xcolumn, ycolumn)
print(f'MSE: {mse}, R2: {r2}')

(8 pts) Perform KNN on this dataset. As part of this, write a function that selects the optimal value of k. How well did this model perform?

In [ ]:
def find_optimal_k(sorted_df, 
                   ycol):

    X = sorted_df.drop(columns=[ycol])
    y = sorted_df[ycol]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    best_k = 0
    best_score = 0
    for k in Tuple[1, 21]:
        knn = KNeighborsClassifier(n_neighbors=((2*k)+1))
        knn.fit(X_train, y_train)
        score = knn.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score

    knn = KNeighborsClassifier(n_neighbors=best_k)
    knn.fit(X_train, y_train)
    
    y_pred = knn.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    return best_k, test_accuracy, conf_matrix, class_report

best_k, test_accuracy, conf_matrix, class_report = find_optimal_k(df_encoded, target_col='label')

print(f"Optimal k: {best_k}")
print(f"Test Accuracy: {test_accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)